## Viewing Behavior Analysis (E)

Goal here is to load in - for every participant - the list with banner-to-position assignments, and merge this with the banner-looked-at list


In [1]:
import os, glob, warnings
import numpy as np
import pandas as pd
warnings.filterwarnings('ignore')


In [2]:
# list all subjects, wich are named sub***
# we have n = 40 
folders = glob.glob("../data/00_gaze_data/sub*")
folders.sort()
subjs = []
for f in folders:
    subjs.append(f[-6:])

print(len(subjs))
subjs[:3]

40


['sub001', 'sub002', 'sub003']

In [3]:
# this is the main piece of code that parses the different ingredient lists:
# - the list that assigns the billboard images to the billboards along the highway
# - the file with the behavioral data (from the VR/eyetracker), containing info about whether a bb was viewed or not

for i in range(len(subjs)):
    print(subjs[i])

    # assemble filepaths for the subjects
    curr_sub_position_file = '../data/00_gaze_data/' + subjs[i] +'/' + subjs[i] +'_billboard_list_position_name_assignment.csv'
    curr_sub_eye_file      = '../data/00_gaze_data/' + subjs[i] +'/' + subjs[i] +'_tracking_data_trial_1.txt'
    out_file_name          = '../data/00_gaze_data/fixation_billboards/' + subjs[i] +'_fixations_billboards.csv'

    #read in position-billboard assignment
    pos_df = pd.read_csv(curr_sub_position_file)

    for row in pos_df.itertuples():
        pos_df.at[row.Index, "billboard_image_at_position_text"] = pos_df.at[row.Index, "billboard_image_at_position"].replace("resources/billboards/","")[:-4]

    pos_df.head()

    #read in fixation information
    eye_df = pd.read_csv(curr_sub_eye_file, sep = '\t')
    eye_df = eye_df.drop(['Unnamed: 9'], axis=1)
    eye_df.head()

    #select fixations and count them
    contain_values = eye_df[eye_df['flag    '].str.contains('fixated')]

    for row in contain_values.itertuples():
        contain_values.at[row.Index, "fix_bill"] = contain_values.at[row.Index, "flag    "].replace("gaze fixated: billboard_image_","")
    contain_values.head()

    print(contain_values['fix_bill'].values)

    #contain_values['fix_bill'].value_counts()

    df = pd.DataFrame(contain_values['fix_bill'].value_counts())
    df['billboard'] = df.index

    df = df.reset_index(level=0)
    #df = df.drop(['index'], axis=1)
    df = df.apply(pd.to_numeric)

    df = df.sort_values(by='billboard')
    df.reset_index(drop=True, inplace=True)
    df.head()
    
    
    zeros = np.zeros(20)
    numlist = np.linspace(1,20,20)
    num_df = pd.DataFrame(np.vstack((zeros, numlist)).T, columns = ['zeros', 'bbnums'])
    num_df.head()

    merged_df = num_df.merge(df, left_on='bbnums', right_on='billboard', how = 'left')
    merged_df['billboard'] = merged_df['bbnums']
    merged_df = merged_df.fillna(0)
    merged_df.head()

    # merge everything together
    merged_df2 = pd.merge(merged_df, pos_df, left_on = 'billboard', right_on = 'position_index')
    merged_df2 = merged_df2.drop(['billboard_image_at_position', 'index'], axis=1)
    merged_df2.to_csv(out_file_name)
    print(merged_df.head())
    print('---')

sub001
['2' '13' '13' '17']
   zeros  bbnums  fix_bill  count  billboard
0    0.0     1.0       0.0    0.0        1.0
1    0.0     2.0       2.0    1.0        2.0
2    0.0     3.0       0.0    0.0        3.0
3    0.0     4.0       0.0    0.0        4.0
4    0.0     5.0       0.0    0.0        5.0
---
sub002
['1' '1' '1' '1' '2' '3' '3' '4' '4' '5' '5' '6' '7' '8' '9' '10' '10'
 '11' '11' '12' '12' '13' '13' '14' '15' '15' '16' '16' '17' '17' '17'
 '18' '19' '19' '19' '20' '20']
   zeros  bbnums  fix_bill  count  billboard
0    0.0     1.0         1      4        1.0
1    0.0     2.0         2      1        2.0
2    0.0     3.0         3      2        3.0
3    0.0     4.0         4      2        4.0
4    0.0     5.0         5      2        5.0
---
sub003
['1' '1' '1' '3' '5' '8' '8' '8' '9' '9' '11' '17']
   zeros  bbnums  fix_bill  count  billboard
0    0.0     1.0       1.0    3.0        1.0
1    0.0     2.0       0.0    0.0        2.0
2    0.0     3.0       3.0    1.0        3.0
3   

In [4]:
merged_df.head()

,zeros,bbnums,fix_bill,count,billboard
0,0.0,1.0,1.0,1.0,1.0
1,0.0,2.0,2.0,1.0,2.0
2,0.0,3.0,3.0,1.0,3.0
3,0.0,4.0,4.0,1.0,4.0
4,0.0,5.0,5.0,2.0,5.0
